In [1]:
!pip install dateparser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 292 kB 5.2 MB/s 


In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import requests
from bs4 import BeautifulSoup 
import re 
import time
import dateparser

In [3]:
cats = ["Societe", "International", "Eco", "Paris", "Culture", "Sport", "Economie"]

In [4]:
cats[1]

'International'

In [5]:
data = requests.get(f"https://www.lejdd.fr/{cats[1]}")

In [6]:
data

<Response [200]>

In [7]:
data.text

'<!DOCTYPE html><!--[if IE 8]><html class="ie8" lang="fr"><![endif]--><!--[if IE 9]><html class="ie9" lang="fr"><![endif]--><html lang="fr"><head><meta charset="utf-8"><meta http-equiv="X-UA-Compatible" content="IE=edge"><link rel="canonical" href="https://www.lejdd.fr/International"><meta name="viewport" content="width=device-width, initial-scale=1"><title>Actualités Internationales | JDD</title><meta property="og:site_name" content="lejdd.fr" /><meta property="og:title" content="Actualités Internationales | JDD" /><meta property="og:description" content="Suivez l&#039;actualité internationale et toute l&#039;info dans le monde avec le JDD. Découvrez tous nos décryptages d&#039;actu." /><meta name="dailymotion-domain-verification" content="dm900myw17x50ert1" /><meta property="fb:app_id" content="373840912675963" /><meta name="author" content="Le JDD" /><meta property="og:locale" content="fr_FR" /><meta name="robots" content="index,follow,max-snippet:-1,max-image-preview:large,noarchiv

In [ ]:
from IPython.core.display import display, HTML
display(HTML(data.text))

In [9]:
html_text = BeautifulSoup(data.text, "html5lib")

In [10]:
link_to_artical = html_text.find_all('h3')[0]

In [11]:
link_to_artical

<h3 class="titre"><span class="nota article_abonnes" style="background-color: #ffc700">
            ABONNÉS
            <i class="icon-logo-JDD-tags"></i></span><a href="/International/oleksiy-arestovytch-conseiller-a-la-presidence-ukrainienne-cette-guerre-sera-terminee-lete-prochain-4151759">Oleksiy Arestovytch, conseiller à la présidence ukrainienne : « Cette guerre sera terminée l’été prochain »</a></h3>

In [12]:
link_to_artical("a")[0]["href"]

'/International/oleksiy-arestovytch-conseiller-a-la-presidence-ukrainienne-cette-guerre-sera-terminee-lete-prochain-4151759'

In [13]:
relative_link = link_to_artical("a")[0]["href"]
relative_link

'/International/oleksiy-arestovytch-conseiller-a-la-presidence-ukrainienne-cette-guerre-sera-terminee-lete-prochain-4151759'

In [14]:
f"https://www.lejdd.fr{relative_link}"

'https://www.lejdd.fr/International/oleksiy-arestovytch-conseiller-a-la-presidence-ukrainienne-cette-guerre-sera-terminee-lete-prochain-4151759'

In [15]:
artical_data = requests.get(f"https://www.lejdd.fr{relative_link}")

In [16]:
artical_data

<Response [200]>

In [ ]:
display(HTML(artical_data.text))

In [18]:
article_bs = BeautifulSoup(artical_data.text, 'html5lib')
article_bs.find_all("p")

[<p class="intro">Oleksiy Arestovytch, blogueur charismatique et conseiller très influent de Volodymyr Zelensky raconte au JDD les prochaines batailles à livrer pour l'armée ukrainienne, fait le point sur l'armement et la crise énergétique. </p>,
 <p>Dans cette <a href="https://www.lejdd.fr/Tags/Ukraine" target="_self">Ukraine</a> en guerre, Oleksiy Arestovytch est l’un des personnages incontournables de la scène politico médiatique. Blogueur charismatique dont la chaîne YouTube compte 1,6 millions d’abonnés, il est devenu un conseiller très influent du Président Volodymyr Zelensky. Celui qui fut autrefois comédien, psychologue et même agent des renseignements est désormais chargé de la propagande militaire et intervient quotidiennement sur les réseaux sociaux pour remonter le moral de la population. Pour le JDD, il évoque les prochaines batailles que devra livrer l’armée ukrainienne et revient sur la crise énergétique que traverse actuellement son pays.</p>,
 <p></p>,
 <p> <strong>Fau

In [19]:
article_bs.find_all("p")[0].text

"Oleksiy Arestovytch, blogueur charismatique et conseiller très influent de Volodymyr Zelensky raconte au JDD les prochaines batailles à livrer pour l'armée ukrainienne, fait le point sur l'armement et la crise énergétique.\xa0"

In [20]:
artical_texts = [p.text for p in article_bs.find_all("p")]
print("\n".join(artical_texts))

Oleksiy Arestovytch, blogueur charismatique et conseiller très influent de Volodymyr Zelensky raconte au JDD les prochaines batailles à livrer pour l'armée ukrainienne, fait le point sur l'armement et la crise énergétique. 
Dans cette Ukraine en guerre, Oleksiy Arestovytch est l’un des personnages incontournables de la scène politico médiatique. Blogueur charismatique dont la chaîne YouTube compte 1,6 millions d’abonnés, il est devenu un conseiller très influent du Président Volodymyr Zelensky. Celui qui fut autrefois comédien, psychologue et même agent des renseignements est désormais chargé de la propagande militaire et intervient quotidiennement sur les réseaux sociaux pour remonter le moral de la population. Pour le JDD, il évoque les prochaines batailles que devra livrer l’armée ukrainienne et revient sur la crise énergétique que traverse actuellement son pays.

 Faut-il s’attendre à ce que la Russie lance de nouvelles salves de missiles sur l’Ukraine ?Oui et cela peut arriver à t

In [ ]:
class JDD:

    def __init__(self):
        self.articals = {}
        self.dates = []
        
    def download_arc_names(self, categorie = 'all', quantity = 50):
        n = int(round(quantity/50+0.5))
        #выгружаем страницы
        categories = 'Politique, Societe, International, Economie, JDD-Paris, Culture, Sport, Sondages'.split(', ')

        if categorie=='all':
            pages=[]
            for cat in categories:
                page_ = [requests.get("https://www.lejdd.fr/"+cat+'/'+str(page_number)) for page_number in range(1,n)]
                pages.extend(page_)
                print(pages)
        else:
            pages = [requests.get("https://www.lejdd.fr/"+categorie+'/'+str(page_number)) for page_number in range(1,n)]
            
        # список названий статей и ссылок
        titles=[]
        for page in pages:
            one_page_ts=BeautifulSoup(page.text, "html5lib").find_all('h3')
            titles.extend(one_page_ts)

        links=["https://www.lejdd.fr"+l("a")[0]["href"] for l in titles]
        n_l=dict(zip([i.text for i in titles], links))
        return n_l
    
    def get_art(self, categorie = 'all', quantity = 50):
        # время печати и тексты статей 
        texts = []
        n_l = self.download_arc_names(categorie, quantity)
        for name, i in list(n_l.items()):
            resp = requests.get(i)
            #time.sleep(0.3)
            bs = BeautifulSoup(resp.text, 'html5lib')

            t = BeautifulSoup(resp.text, "html5lib").find_all('time')[0]
            t = ' '.join(t.get_text().split(',')[:2])
            time = dateparser.parse(t)

            text = BeautifulSoup(" ".join([p.text for p in bs.find_all("p")]), "html5lib").get_text().replace('\xa0', ' ')
        
            self.articals[time] = (name, text)
            self.dates.append(time)    
        self.dates.sort()
        print('Corpus of {} articals is done. First article published {}, the last one  - {}'.format(len(self.dates), min(self.dates),max(self.dates)))
        return self.articals
            
    def choose_time_per(self, l_b, r_b):
        l_b = dateparser.parse(l_b)
        r_b = dateparser.parse(r_b)
        for date in self.dates:
            if date >= l_b and date <= r_b :
                self.subcorpus[date] = (self.articals[date][0], self.articals[date][1])
        return self.subcorpus
    
    def clear_subcorpus(self):
        self.subcorpus= []
    
    def write_coprus(self, corpus):
        
        f = open('all_articals.txt', 'w', encoding='utf-8')
        for date,(name, text) in corpus.items():
            f.write(str(date)+'\n')
            f.write(name+'\n')
            f.write(text+'\n')
        f.close()
        print('Corpus({} articls) is written in file all_articals.txt'.format(len(list(corpus.items()))))

 

In [ ]:
jdd = JDD()
jdd.get_art(categorie='Politique', quantity=50)
jdd.write_coprus(jdd.articals)

Corpus of 50 articals is done. First article published 2022-02-19 19:15:00, the last one  - 2022-02-26 00:00:00
Corpus(44 articls) is written in file all_articals.txt


In [ ]:
!cat all_articals.txt

2022-02-26 00:00:00
C'est quoi la police de proximité? 
LES CLES DE L'ELYSEE. Jusqu'à la fin de la campagne présidentielle, l'antisèche du Journal du Dimanche va définir les mots du débat politique. Parce qu'il n'y a jamais de questions bêtes, notre rubrique va revenir sur les notions utilisées par tous les candidats et les commentateurs sans que l'on prenne jamais le temps de les expliquer. Quand on parle de police de proximité, ce n'est pas de police municipale dont on parle, mais d’une façon de concevoir l'action de la police. La police de proximité, c'est ce qu'on appelle une "doctrine d’emploi", de la police nationale, qui a été mise en place sous le gouvernement de Lionel Jospin, par son ministre de l'Intérieur, Jean-Pierre Chevènement, en 1998.  Elle s'inspire d'un modèle anglo-saxon le community policing. Avec la police de proximité, le gouvernement veut renouer le lien entre les forces de l'ordre et la population, et lutter contre le sentiment d’insécurité dans les quartiers. 